# Competitors data

In [5]:
import pandas as pd
import requests


In [6]:
# Look in documentation --> find place
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"
api_key = "" # Your Google API key

# actual api key
location = "Zürich"
supermarkets = ["Lidl", "Aldi", "Denner", "Alnatura", "Spar", "Migros", "Coop"]
postal_codes = [8001, 8002, 8003, 8004, 8005, 8006, 8008, 8032, 8037, 8038, 8041, 8044, 8045, 8046, 8047, 8048, 8049, 8050, 8051, 8052, 8053, 8055, 8057, 8064]

search_terms = []

for supermarket in supermarkets:
    
    for postal_code in postal_codes:
        
        search_term = supermarket + ", " + location + ", " + str(postal_code)
        
        search_terms.append(search_term)

In [7]:
competitors_df = pd.DataFrame()

for search_term in search_terms:
    
    query = search_term
    
    next_page_token = ""
    
    for num_page in range(1, 4):
    
        try:
            req = requests.get(url + "query=" + query + "&key=" + api_key + "&next_page_token=" + "")
        except:
            print("Error in the get")
    
        
        places_json = req.json()
    
        try:
            next_page_token = places_json["next_page_token"]
        except:
            next_page_token = ""
        
        
        results_df = pd.json_normalize(places_json["results"])
        results_df["supermarket_brand"] = search_term
        competitors_df = competitors_df.append(results_df)
        
    competitors_df = competitors_df.drop_duplicates(["formatted_address", "name", "supermarket_brand"]).reset_index(drop=True)
        
    print("{}: OK".format(search_term))

Lidl, Zürich, 8001: OK
Lidl, Zürich, 8002: OK
Lidl, Zürich, 8003: OK
Lidl, Zürich, 8004: OK
Lidl, Zürich, 8005: OK
Lidl, Zürich, 8006: OK
Lidl, Zürich, 8008: OK
Lidl, Zürich, 8032: OK
Lidl, Zürich, 8037: OK
Lidl, Zürich, 8038: OK
Lidl, Zürich, 8041: OK
Lidl, Zürich, 8044: OK
Lidl, Zürich, 8045: OK
Lidl, Zürich, 8046: OK
Lidl, Zürich, 8047: OK
Lidl, Zürich, 8048: OK
Lidl, Zürich, 8049: OK
Lidl, Zürich, 8050: OK
Lidl, Zürich, 8051: OK
Lidl, Zürich, 8052: OK
Lidl, Zürich, 8053: OK
Lidl, Zürich, 8055: OK
Lidl, Zürich, 8057: OK
Lidl, Zürich, 8064: OK
Aldi, Zürich, 8001: OK
Aldi, Zürich, 8002: OK
Aldi, Zürich, 8003: OK
Aldi, Zürich, 8004: OK
Aldi, Zürich, 8005: OK
Aldi, Zürich, 8006: OK
Aldi, Zürich, 8008: OK
Aldi, Zürich, 8032: OK
Aldi, Zürich, 8037: OK
Aldi, Zürich, 8038: OK
Aldi, Zürich, 8041: OK
Aldi, Zürich, 8044: OK
Aldi, Zürich, 8045: OK
Aldi, Zürich, 8046: OK
Aldi, Zürich, 8047: OK
Aldi, Zürich, 8048: OK
Aldi, Zürich, 8049: OK
Aldi, Zürich, 8050: OK
Aldi, Zürich, 8051: OK
Aldi, Züric

In [10]:
competitors_df.head()
print(competitors_df.shape)

(1924, 24)


In [11]:
# Save data
competitors_df.to_csv("supermarkets_raw_data.csv", index=False)

# Data preparation

In [50]:
# Read data
competitors_df = pd.read_csv("supermarkets_raw_data.csv")

# Filter the supermarkets that are "permanently_closed"
competitors_df = competitors_df[competitors_df["permanently_closed"] != True]

# Split column supermarket_brand

competitors_df[["search_name", "search_location", "search_postal_code"]] = competitors_df["supermarket_brand"].str.split(",", expand=True)

# Columns selection
competitors_df = competitors_df[["geometry.viewport.northeast.lat", "geometry.viewport.northeast.lng", "formatted_address", "name", "rating", "search_name", "search_postal_code"]]

# Check for outliers names
competitors_df["name"].unique()
names_filter = ['Billett-Service Migros', 'SUN STORE Migros City Zürich', 'Migros Florissimo', 'VOI Migros Partner','Migros-Genossenschafts-Bund Group IT',
                 'Do it + Garden - City Farming Zürich-Oerlikon', 'melectronics', 'Migros Electronics', 'VOI', 'REFORMHAUS', 'DHL Service Point (SPAR EXPRESS)', 'Postal agency 8068 Zürich West',
                 'DHL Service Point (Spar Supermarkt Zollfreilager)', 'Dürr A. & CO AG', 'Klubschule Migros Zürich Oerlikon', 'Do it + Garden Zurich - Brunaupark', 'SportXX', 'Neumarkt Oerlikon',
                'Coop Bau+Hobby Zürich Oerlikon Eleven', 'Reto Suhner']

# Filter outliers
competitors_df = competitors_df[~competitors_df.name.isin(names_filter)]

# Change name to upper
competitors_df["name"] = competitors_df["name"].str.upper()
competitors_df["search_name"] = competitors_df["search_name"].str.upper()


competitors_df = competitors_df[competitors_df.apply(lambda x: x['search_name'] in x['name'], axis = 1)]
competitors_df = competitors_df.drop_duplicates(["formatted_address", "name", "search_name"]).reset_index(drop=True)

# Check columns
# competitors_df = competitors_df[["geometry.viewport.northeast.lat", "geometry.viewport.northeast.lng", "formatted_address", "main_name", "rating", "supermarket_name", "postal_code"]]

competitors_df.head(10)

,geometry.viewport.northeast.lat,geometry.viewport.northeast.lng,formatted_address,name,rating,search_name,search_postal_code
0,47.370288,8.542742,"Fraumünsterstrasse 16, 8001 Zürich, Switzerland",LIDL SWITZERLAND,4.4,LIDL,8001
1,47.363033,8.516097,"Grubenstrasse 28, 8045 Zürich, Switzerland",LIDL SCHWEIZ,4.1,LIDL,8002
2,47.393540,8.516621,"Förrlibuckstrasse 62, 8005 Zürich, Switzerland",LIDL SCHWEIZ,4.3,LIDL,8002
3,47.411778,8.547188,"Ohmstrasse 11, 8050 Zürich, Switzerland",LIDL SCHWEIZ,4.1,LIDL,8002
4,47.411924,8.543998,"Hofwiesenstrasse 350, 8050 Zürich, Switzerland",LIDL SCHWEIZ,3.9,LIDL,8002
5,47.409028,8.586408,"Ueberlandstrasse 381, 8051 Zürich, Switzerland",LIDL,4.1,LIDL,8002
6,47.423829,8.500618,"Wehntalerstrasse 625, 8046 Zürich, Switzerland",LIDL SCHWEIZ,4.2,LIDL,8002
7,47.400842,8.434019,"Reitmenstrasse 5, 8952 Schlieren, Switzerland",LIDL,4.1,LIDL,8002
8,47.413644,8.614931,"Neue Winterthurerstrasse 2, 8304 Wallisellen, ...",LIDL,4.1,LIDL,8002
9,47.456127,8.580686,"Schaffhauserstrasse 165, 8302 Kloten, Switzerland",LIDL,4.1,LIDL,8002


In [48]:
# Save data
competitors_df.to_csv("supermarkets_clean_data.csv", index=False)